# Declarative Data Visualisation
This strategy is "declarative" in nature. Instead of writing a "program" in the traditional sense, we write instead the "spec" which is fed to an interpreter. The interpreter deals with the assorted mathematics, getting lines on screens etc. Examples of this approach include, but are not limited to 

- [Vega](https://vega.github.io)
- [Vega lite](https://vega.github.io/vega-lite)
- [Apache E-charts](https://echarts.apache.org/en/index.html)
- [Plotly](https://plotly.com/graphing-libraries/)

Co-incidentally, all of these libraries, use JSON as the "spec format", or transport.

In [2]:
private val spec = ujson.read("""{
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "A simple bar chart with embedded data.",
    "data": {
      "values": [
        {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "a", "type": "nominal", "axis": {"labelAngle": 0}},
      "y": {"field": "b", "type": "quantitative"}
    },
    "width" : 300
}
""")

spec("data")("values") = ujson.Arr(
  ujson.Obj("a" -> "A", "b" -> -10),
  ujson.Obj("a" -> "A", "b" -> -10),
  ujson.Obj("a" -> "A", "b" -> -10)
)


kernel.publish.display(
  almond.interpreter.api.DisplayData(
        data = Map(
          "application/vnd.vega.v5+json" -> ujson.write(spec)
        )
  )
)

Hopefully, you've created a bar plot, which looks like this.

![Bar](pictures/Simple_Bar.png)
## So, what happened?

There are really only two scala statements here:
1. Declare a cunningly constructed string
2. Feed it to the almond kernel 

Key Points: 
- You'll note that if you feed Jupyter the right mime-type... it will 
    - treat your string as a chart 
    - wheel in the vega libraries and generally manage the meta problems of getting all that infrastructure in the right place. Rather convienient. 
- At first glance, this "feed the kernel a mime type" looks a bit brittle. Importantly though, this is the key part of how many visualisation libraries work under the hood in Jupyter. One famous example is [Altair](https://altair-viz.github.io/getting_started/overview.html), one of the marquee python data science plotting libraries. So this is a surprinsingly robust solution, that works across many Jupyter frontends
- That "cunningly constructed" string, looks a lot like JSON...

